
Author : Chadha SAKKA (M2 Saclay CHPS)

# LAB 3 : Non-Blocking Algorithms
##Exercice 4 : A simple memory allocator

1. **The definition of a Memory Allocator**
2. **The Chunk Structure**
3. **Allocation and Deallocation mechanism**
4. **The purpose of using Locks in multithreaded nvironments**
5. **Implementing the Lock-Based Allocator**
6. **Introduction to Lock-Free Programming**
7. **Implementing the Lock-Free Allocator**
8. **Conclusion**

---

### 1. What is a Memory Allocator?

A **memory allocator** is a system that manages dynamic memory allocation for a program. In C, functions like `malloc` and `free` are provided by the C standard library to allocate and deallocate memory at runtime.

In some cases, we might want to implement our own memory allocator to have more control over how memory is allocated and freed. This can be useful for optimizing performance, implementing custom memory management strategies, or managing resources in a multithreaded environment.

---

### 2. The Chunk Structure

In our allocator, we will manage memory in units called **chunks**. Each chunk represents a contiguous block of memory that can be either **allocated** or **free**.

Here's the structure of a chunk  :

```c
struct chunk {
  size_t size; // The size of the memory block respresnted by the chunk
  union {
    struct chunk* next; //Used when the chunk is free */
    char content[0];    // used when the chunk is allocated */
  };
};

```
---

### 3. Allocation and Deallocation Mecanism

#### Allocation (`alloc_chunk(size_t size)`):

- **Goal**: Find or create a chunk of at least `size` bytes that we can give to the user.

- **Process**:

  1. **Search the Free List**: Look through the `free_list` to find a chunk `c` such that `c->size >= size`. This means the chunk is big enough to satisfy the allocation request.

  2. **Remove the Chunk**: Once we find a suitable chunk, we remove it from the `free_list` since it's no longer free.

  3. **Split the Chunk (if necessary)**: If the chunk is significantly larger than needed, we can split it into two:

     - The first part (`c`) will be of the requested `size` and will be returned to the user.
     - The second part (`d`) will be the remaining memory, which we can add back to the `free_list` for future allocations.

  4. **Return the Chunk**: We return the allocated chunk to the user.

- **Creating a New Chunk**: If no suitable chunk is found in the `free_list`, we create a new large chunk (e.g., 64 MB) and add it to the `free_list`, then retry the allocation.

#### Deallocation (`free_chunk(struct chunk* c)`):

- **Goal**: Add the chunk `c` back to the `free_list` so that it can be reused in future allocations.

- **Process**:

  1. **Insert into Free List**: We add the chunk to the front of the `free_list`.

---

### 4. The purpose of Using Locks in Multithreaded Environments

In a multithreaded program, multiple threads might try to allocate or free memory at the same time. This can lead to **race conditions** where the `free_list` is modified by multiple threads simultaneously, resulting in inconsistent or corrupted data.

To prevent this, we use **locks** (such as a `pthread_mutex_t` mutex) to ensure that only one thread can modify the `free_list` at a time. This is known as **mutual exclusion**.

---

### 5. Implementing the Lock-Based Allocator

#### 5.1. Give the code of `void free_chunk(struct chunk* c)` , which adds c to free_list.

```c
#include <pthread.h>
// Global variables
pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER; // Global mutex lock
struct chunk* free_list = NULL;                    // Global free list

// Function to add a chunk back to the free list
void free_chunk(struct chunk* c) {
    pthread_mutex_lock(&mutex);  // Acquire the lock for thread safety
    c->next = free_list;         // Pointing c->next to the current head of free_list
    free_list = c;               // Making c the new head of free_list
    pthread_mutex_unlock(&mutex); // Release the lock
}ock
}
```

**Explanation:**

- We lock the mutex before modifying `free_list` to prevent other threads from interfering.
- We insert `c` at the beginning of `free_list` by setting `c->next` to the current `free_list` and updating `free_list` to point to `c`.
- We unlock the mutex after we're done.

#### 5.2. Give the code of `struct chunk* create_large_chunk()`
This function allocates a large chunk of 64*1024*1024 bytes, adequatly initializes its size field and returns it. It doesn't add the new chunk in the list: the function simply returns the new chunk. To allocate the chunk, you just have to use malloc.

```c
#include <stdlib.h>

#define LARGE_CHUNK_SIZE (64 * 1024 * 1024) // 64 MB

struct chunk* create_large_chunk() {
    size_t total_size = sizeof(struct chunk) + LARGE_CHUNK_SIZE;
    struct chunk* c = (struct chunk*)malloc(total_size); // Allocate memory

    if (!c) {
        // Handle allocation failure if necessary
        return NULL;
    }

    c->size = LARGE_CHUNK_SIZE; // Initialize the size of the chunk
    return c; // Return the new chunk
}
```

**Explanation:**

- We allocate a large block of memory using `malloc`, including space for the `struct chunk` header.
- We initialize the `size` field to `LARGE_CHUNK_SIZE` to indicate how much usable memory is in the chunk.
- We return the new chunk.

#### 5.3.  Give the code of `struct chunk* alloc_chunk(size_t size)`. If free_list does not contain any chunk larger than c, then, alloc_chunk uses create_large_chunk to create a new large chunk. In this case, alloc_chunk adds the new large chunk to the free_list, and restarts to try to allocate a chunk.

```c
struct chunk* alloc_chunk(size_t size) {
    pthread_mutex_lock(&mutex); // Acquire the lock

    struct chunk** prev = &free_list;
    struct chunk* c = free_list;

    // 1) Find the first suitable chunk in thefree_list
    while (c) {
        if (c->size >= size) {
            *prev = c->next; // Remove c from free_list
            break;
        }
        prev = &c->next;
        c = c->next;
    }

    if (!c) {
        // No suitable chunk found
        pthread_mutex_unlock(&mutex); // Release the lock before allocating
        c = create_large_chunk();     // Create a new large chunk
        if (!c) {
            // Allocation failed
            return NULL;
        }
        // Add the new chunk to free_list
        pthread_mutex_lock(&mutex);   // Re-acquire the lock
        c->next = free_list;
        free_list = c;
        pthread_mutex_unlock(&mutex); // Release the lock
        // Retry allocation
        return alloc_chunk(size);
    }

    // 2) Deciding if we need to split the chunk
    if (size + sizeof(struct chunk) < c->size) {
        // if the chunk is larger than needed then split the chunk
        struct chunk* d = (struct chunk*)((uintptr_t)c + sizeof(struct chunk) + size);
        d->size = c->size - size - sizeof(struct chunk); // Remaining size
        c->size = size; // Adjust the size of the allocated chunk

        // Free the remaining chunk
        d->next = free_list;
        free_list = d;
    }

    pthread_mutex_unlock(&mutex); // Release the lock
    return c; // Return the allocated chunk
}
```

- **Finding a Suitable Chunk**:

  - We traverse the `free_list` to find a chunk `c` where `c->size >= size`.
  - We use `prev` to keep track of the previous chunk so that we can remove `c` from the list when found.
  
- **Handling No Suitable Chunk**:

  - If no suitable chunk is found, we unlock the mutex and create a new large chunk.
  - We lock the mutex again to add the new chunk to the `free_list` and then unlock it.
  - We call `alloc_chunk(size)` again to retry the allocation.
  
- **Splitting the Chunk**:

  - If the chunk is larger than needed (i.e., `size + sizeof(struct chunk) < c->size`), we split it.
  - We calculate the address of the new chunk `d` by advancing the pointer `c` by `sizeof(struct chunk) + size`.
  - We adjust the sizes of `c` and `d`.
  - We add `d` back to the `free_list` for future allocations.
  
- **Returning the Allocated Chunk**:

  - We unlock the mutex and return the allocated chunk `c`.


---



### 6. Introduction to Lock-Free Programming

In lock-based programming, we use locks to prevent concurrent access to shared resources. However, locks can cause performance issues due to contention and can lead to **deadlocks** if not used carefully.

In **Lock-free programming** we use atomic operations to ensure that concurrent access to shared data structures is done safely without using locks. This can improve performance and avoid issues like deadlocks.

- **Atomic Operations**: Operations that are completed in a single step from the perspective of other threads. They cannot be interrupted by other threads.

- **Memory Orderings**: Specifies how memory operations are ordered around atomic operations to ensure proper synchronization between threads.

---

### 7. Implementing the Lock-Free version Allocator

To implement the lock-free version, we'll use atomic operations provided by C11's `<stdatomic.h>` library.


#### 7.0. Defining a custom `chunk_lock_free` :
```c
#include <stdatomic.h>

struct chunk_lock_free {
    size_t size;
    union {
        _Atomic(struct chunk_lock_free*) next; // Atomic next pointer
        char content[0];
    };
};
```
#### 7.1. Atomic Free List Declaration

```c
_Atomic(struct chunk_lock_free*) free_list = NULL;
```

**Explanation:**

- We declare `free_list` as an atomic pointer to a `struct chunk`, ensuring that all accesses to it are atomic.

#### 7.2. `struct chunk* pop()`

```c
// Atomic pop operation
static struct chunk_lock_free* pop() {
    struct chunk_lock_free* head;

    while (1) {
        head = atomic_load_explicit(&free_list, memory_order_acquire);

        if (head == NULL) {
            return NULL; // The free list is empty
        }

        struct chunk_lock_free* next = atomic_load_explicit(&head->next, memory_order_relaxed);

        if (atomic_compare_exchange_weak_explicit(
                &free_list,
                &head,
                next,
                memory_order_acquire,
                memory_order_relaxed)) {
            // Successfully removed the head; return it
            return head;
        }
        // Retry if the operation failed
    }
}
```

**Explanation:**

- **Purpose**: Atomically removes and returns the head of the `free_list`.

- **Process**:

  - We use `atomic_load_explicit` to get the current `free_list` value.
  - If `free_list` is `NULL`, the list is empty, and we return `NULL`.
  - We attempt to update `free_list` to `head->next` using `atomic_compare_exchange_weak_explicit`.
    - This function attempts to set `free_list` to `next` **only if** `free_list` is still equal to `head`.
    - If another thread has modified `free_list` between our load and compare-exchange, the operation fails, and we retry.
  - Memory orderings (`memory_order_acquire` and `memory_order_release`) ensure proper synchronization.

#### 7.3. `void push(struct chunk_lock_free* c)`

```c
void push(struct chunk_lock_free* c) {
    struct chunk_lock_free* old_head;
    do {
        old_head = atomic_load_explicit(&free_list, memory_order_relaxed);
        atomic_store_explicit(&c->next, old_head, memory_order_relaxed);
    } while (!atomic_compare_exchange_weak_explicit(
        &free_list,
        &old_head,
        c,
        memory_order_release,
        memory_order_relaxed));
}
```

**Explanation:**

- **Purpose**: Atomically adds a list of chunks from `head` to `tail` to the `free_list`.

- **Process**:

  - We load the current `free_list` into `old_head`.
  - We set `tail->next` to `old_head` to link our list into the current `free_list`.
  - We attempt to update `free_list` to `head` using `atomic_compare_exchange_weak_explicit`.
    - If the compare-exchange succeeds, our list is now at the front of `free_list`.
    - If it fails, `old_head` is updated, and we retry.
  - By having both `head` and `tail`, we avoid the need to traverse the list to find the end.

#### 7.4. `void free_chunk(struct chunk* c)`

```c
void free_chunk_lock_free(struct chunk* c_generic) {
    struct chunk_lock_free* c = (struct chunk_lock_free*)c_generic;
    push(c);
}
```

**Explanation:**

- We reuse the `push` function to add a single chunk back to the `free_list`.
- `c` is both the head and the tail of the list we're pushing.

#### 7.5. `struct chunk* alloc_chunk_lock_free(size_t size)`

```c
struct chunk* alloc_chunk_lock_free(size_t size) {
    struct chunk_lock_free* c;

    while (1) {
        c = pop();
        if (!c) {
            // No chunks available, create a new large chunk
            c = create_large_chunk();
            if (!c) {
                // Allocation failed
                return NULL;
            }
            // Add the new chunk to free_list and retry
            free_chunk_lock_free((struct chunk*)c);
            continue;
        }

        if (c->size >= size) {
            // Decide whether to split the chunk
            if (size + sizeof(struct chunk_lock_free) < c->size) {
                // Split the chunk
                uintptr_t chunk_address = (uintptr_t)c;
                uintptr_t new_chunk_address = chunk_address + sizeof(struct chunk_lock_free) + size;

                // Ensure alignment
                new_chunk_address = (new_chunk_address + alignof(struct chunk_lock_free) - 1) & ~(alignof(struct chunk_lock_free) - 1);

                struct chunk_lock_free* d = (struct chunk_lock_free*)new_chunk_address;
                d->size = c->size - size - (new_chunk_address - chunk_address);
                c->size = size; // Adjust the size of the allocated chunk

                // Free the remaining chunk
                free_chunk_lock_free((struct chunk*)d);
            }
            // Return the allocated chunk
            return (struct chunk*)c;
        } else {
            // Chunk too small, add it back to free_list
            free_chunk_lock_free((struct chunk*)c);
            // Loop again to try another chunk
        }
    }
}
```


**Explanation of the Lock-Free Allocator Code**

The `alloc_chunk_lock_free(size_t size)` function implements a lock-free memory allocator using atomic operations to manage the free list (`free_list`) in a thread-safe manner. Here's a summary of how it works:

1. **Main Loop**: The function enters an infinite loop to continuously attempt to allocate a suitable chunk.

2. **Popping from the Free List**:
   - It calls `pop()` to atomically remove the first chunk from the `free_list`.
   - If `pop()` returns `NULL`, it means the free list is empty.

3. **Handling an Empty Free List**:
   - If no chunks are available, the function creates a new large chunk of 64 MB by calling `create_large_chunk()`.
   - This new chunk is immediately freed via `free_chunk_lock_free()` to be added to the `free_list`.
   - The loop continues to retry with the updated `free_list`.

4. **Checking Chunk Size**:
   - If the retrieved chunk is large enough (`c->size >= size`), the function decides whether to split it.
   - If the chunk is too small, it's returned to the `free_list` via `free_chunk_lock_free(c)`, and the loop continues.

5. **Splitting the Chunk (if necessary)**:
   - If splitting is feasible (`size + sizeof(struct chunk_lock_free) < c->size`), the chunk is divided into two:
     - The chunk `c` is adjusted to the requested size.
     - A new chunk `d` is created for the remaining space, properly aligned, and added to the `free_list`.

6. **Returning the Allocated Chunk**: The chunk `c`, now resized appropriately, is returned to the caller.

##**Analysis of the Obtained Results**

The `Lock-Based Allocator` takes **0.38 seconds**, and the `Lock-Free Allocator` takes **0.46 seconds**. Additionally, ThreadSanitizer detected **data races** in the implementation of the lock-free allocator.

##**Interpreting the Data Races Detected**

The warnings from ThreadSanitizer indicate that threads are concurrently accessing shared memory regions without proper synchronization, leading to undefined behavior. Specifically, the messages point to concurrent access to the `next` field of chunks.

- **Location of the Issue**:
  - The data races occur on the same chunk in memory, indicating that multiple threads are manipulating the same chunk without adequate synchronization.

**Why Are These Data Races Occurring?**

- **Concurrent Initialization**:
  - When splitting a chunk, the new chunk `d` is created, and its `next` field may be modified by one thread while another thread could access it.
  - If chunk `d` is added to the `free_list` before its initialization is complete, another thread might retrieve it via `pop()` and access uninitialized data.

- **Lack of Synchronization During Splitting**:
  - The initialization of the split chunk `d` isn't protected, allowing other threads to access it before initialization is finished.

##**How I managed These Data Races**

We need to make sure that the initialization of new chunks is complete before they become accessible to other threads. Possible solutions include:

1. **Ensure Complete Initialization Before Adding to `free_list`**:
   - Before calling `free_chunk_lock_free((struct chunk*)d);`, make sure all fields of `d` are fully initialized.
   - We use a memory barrier to guarantee that writes are visible to other threads:
     ```c
     atomic_thread_fence(memory_order_release);
     ```

2. **Use Atomic Operations for Shared Fields**:
   - Declare the `size` field as atomic if multiple threads can access it simultaneously:
     ```c
     _Atomic size_t size;
     ```
   - This ensures that reads and writes to `size` are atomic, preventing data races.

3. **Protect Access to the `next` Field**:
   - Ensure that all modifications to the `next` field are performed using appropriate atomic operations.
   - If `next` is already an atomic pointer, verify that all reads and writes use the correct atomic functions.

4. **Verify Alignment and Integrity of Chunks**:
   - When splitting, ensure that the new chunk `d` is correctly aligned and its size is valid.
   - Avoid using misaligned pointers or negative sizes, which could cause undefined behavior.

5. **Use Safe Initialization Sequences**:
   - Initialize all fields of chunk `d` before adding it to the `free_list`.
   - For example:
     ```c
     d->size = new_size;
     atomic_store_explicit(&d->next, NULL, memory_order_relaxed);
     ```
     - Use `atomic_store_explicit` to initialize `next` atomically.

6. **Prevent Premature Access to Chunk `d`**:
   - Ensure that no other thread can access chunk `d` until its initialization is fully complete.
   - By making sure all writes are finished before adding `d` to the `free_list`, you prevent other threads from reading partially initialized data.

##**Why Is the Lock-Free Allocator Slower?**

- **Overhead of Atomic Operations**:
  - Atomic operations are generally more expensive than regular memory accesses.
  - In an environment with low contention, the overhead of atomic operations can outweigh the benefits of lock-free execution.

- **Increased Complexity**:
  - Handling special cases like chunk splitting adds complexity and additional checks.
  - This can slow down execution compared to a lock-based allocator with simpler logic.

##**Conclusion**

The lock-free allocator may exhibit slower performance in some cases due to the overhead associated with atomic operations and additional complexity. However, it offers advantages in scalability in high-contention environments.
